In [2]:
import openai
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt  

from azure.core.credentials import AzureKeyCredential 
from azure.search.documents import SearchClient  

from azure.search.documents.models import VectorizedQuery
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [3]:
import json
import os
from dotenv import load_dotenv

In [21]:
load_dotenv("pfd_conf.env")
container_name = "cvjson"  # Name of the container in Blob Storage
blob_name = 'salida.json'  # Name of the JSON file in Blob Storage
blob_vectorname = 'docVectors.json'

connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
service_endpoint = os.getenv("AZURE_STORAGE_SERVICE_ENDPOINT")

index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_KEY")

model = os.getenv("AZURE_EMBEDDINGS_MODEL")
credential = AzureKeyCredential(key)


client_embeddings = AzureOpenAI(
  api_key = os.getenv("OPENAI_EMBEDDINGS_KEY"),  
  api_version = os.getenv("OPENAI_EMBEDDINGS_VERSION"),
  azure_endpoint = os.getenv("OPENAI_EMBEDDINGS_ENDPOINT")
)

endpoint = os.getenv("AZUREOPENAI_ENDPOINT")
deployment = os.getenv("AZUREOPENAI_DEPLOYMENT")

 
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key="51ce1634b07b4fd0b774b30e2d6d8f8b",
    api_version="2024-02-01",
)



# Function to chat with your documents

To chat with your documents you'll need to upload the vectorized documents on azure search

this way it will look for the best machof your query

the the results will the feeded into a gpt model

In [22]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])


In [23]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text, model=model):
    return client_embeddings.embeddings.create(input = [text], model=model).data[0].embedding


In [24]:
def seleccionaTop(text):
    n_ret = 5
    arr_numstring = ["uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve", "diez", "once", "doce", "trece", "catorce", "quince"]
    for i, num_string in enumerate(reversed(arr_numstring), start=1):
        
        if num_string in text or str( (len(arr_numstring) + 1 )- i) in text:
            if int((len(arr_numstring) + 1 ) - i) > 10:
                return False, int((len(arr_numstring) + 1 ) - i)
            if int((len(arr_numstring) + 1 ) - i) < 5:
                return True, 6
            
            return  True, int(((len(arr_numstring) + 1 ) - i) * 1.5)
    
    return True, 6


In [26]:
def azure_cognitive_search(query, top):
    search_client = SearchClient(service_endpoint, index_name, credential=credential)
    embedding = generate_embeddings(query)
    vector_query_content = VectorizedQuery(vector=embedding, 
                                    k_nearest_neighbors=23, 
                                    fields= "contentVector",
                                    exhaustive=True)
    
    results = search_client.search(  
        search_text=query,  
        vector_queries=[vector_query_content],
        select=["Area", "content", "filename"],
        top= top
    )

    mess = ""
    filename = []
    folder = []
    cont_result = 0

    for result in results:
        mess += (
            #f"Area donde trabaja: {result['Area']}\n"
            f"Contenido del CV: {result['content']}\n **** "
        )
        folder.append(f"/Documentos compartidos/General/{result['Area']}")
        filename.append(result['filename'])
    
    return mess, folder, filename




In [31]:

def azure_openai(query, prompt_results = "", isIntro = False):
    if not isIntro:
        
        prompt =[
            {
                "role": "system",
                "content": "Eres un chatbot de inteligencia artificial de la empresa Global Hitss (GH), diseñado para ofrecer información sobre los currículos de los empleados. Cada currículum detalla las diversas habilidades y conocimientos de los empleados. Asegúrate de que la respuesta vaya acorde a la consulta que te están haciendo. Los archivos de curriculum a los que tienes acceso están delimitados por '****', separados por personas. Cada documento contiene los siguientes campos: Nombre completo, Resumen profesional, Resumen de habilidades técnicas, Experiencia laboral, habilidades técnicas, Cursos y certificaciones, educación e Idiomas (o Inglés), estos campos están en orden."
            },
            {
                "role": "system",
                "content": "Para mejorar la comprensión, cada respuesta se presenta en formato estructurado. Siempre Los nombres de las personas se destacan en negritas (Nombre Segundo Nombre y Apellido paterno), esto es lo único que se resalta en negritas. Se inicia siempre con el nombre del colaborador, seguido de una lista con viñetas de sus características relevantes con respecto a la pregunta formulada. Estas características deben estar relacionadas textualmente con la pregunta. Las respuestas deben ser un resumen del contexto de cada colaborador que responda a la pregunta formulada, proporciona la respuesta con el texto que se da de cada colaborador, No debes de mezclar información. SOLO En el caso de que se pregunte por un certificado únicamente deberás de incluir respuestas refrentes al apartado de cursos y certificaciones dentro del texto, no incluyas habilidades, ni habilidades técnicas, ni experiencia. Cuando te pregunten: quien esta certificado en <>?, toma la pregunta como: quien tiene certificación en <>?, solo centrate en el apartado de certificaciones y no inventes información Nunca asumes información, si tienen una certificación por ejemplo de google no quiere decir que tiene certificación en GCP o algo especifico."
            },
            {
                "role": "system",
                "content": "Al evaluar la información, asegúrate de revisarla exhaustivamente e intenta proporcionar el número de resultados solicitados. Intenta obtener la mayor información posible de cada colaborador que responda a la consulta. No se debe inventar información. Para que la pregunta sea considerada, al menos una palabra, generalmente la palabra por la que se pregunta, debe aparecer en el contexto del colaborador, recuerda que en la pregunta pueden faltar subfijos, prefijos, mayusculas o minuscular. Se debe intentar obtener la mayor cantidad de colaboradores posibles que cumplan con los criterios de selección. Si el colaborador no cumple con los criterios de la pregunta, se omitirá cualquier mención del colaborador y no habrá mención. Si la información de un colaborador no es relevante para la pregunta, se excluye el resulatado de ese colaborador y no habrá mención. No debes mezclar experiencias laborales. En caso de que ningún empleado proporcione información útil o relevante, se informa que no se encontraron coincidencias en la búsqueda y se pregunta si tienen alguna otra consulta. Recuerda que la única fuente de conocimiento y de aprendizaje es la que se te otorga como información, las preguntas del usuario no son fondo de conocimiento y no sé debe de tomar como tal, las preguntas nunca proporcionan información adicional al contexto. Si se te pregunta si Una persona sabe de <> y no existe esta habilidad, experiencia o certificación en su curriculim entonces, omitirás a esta persona Nunca inventas información."
            },
            {
                "role": "system",
                "content": f"Información:{prompt_results}"
            },
            {
                "role": "user",
                "content": f"Pregunta o consulta: {query}"
            }
        ]


    deployment = os.get_env("AZUREOPENAI_DEPLOYMENT")
 
    response = client.chat.completions.create(
        model= deployment, 
        
        messages = prompt,
        temperature= 0.3, 
        max_tokens=1000,
        top_p = 0.3,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=None,
        
        n = 4
    )
    
    if(response.choices[0].message.content == "False"):
        return False, response.choices[0].message.model_dump_json(indent=2)
    return True, response.choices[0].message.model_dump_json(indent=2)

    


In [32]:
def selecciona_rutas(cognitive, contenido, folder, filename):
    matches = [remove_accents(match.upper()) for match in re.findall(r'\*\*(.*?)\*\*', contenido, re.DOTALL)]
    print(matches)

    parts = cognitive.split("****") 
    
    sicontiene = []

    for i, part in enumerate(parts[:-1]):   
        siGuarda = False
        part = part[:400]
        for match in matches:
            for j in range(len(part.split()) - len(match.split()) + 1):
                joined_string = remove_accents(" ".join(part.split()[j:j+len(match.split())])).upper()
                vectorizer = CountVectorizer().fit_transform([match, joined_string])
                vectors = vectorizer.toarray()
                cosine_sim = cosine_similarity(vectors)[1][0]
                
                if cosine_sim > 0.60:
                    sicontiene.append(i)
                    siGuarda = True
                    break
            if siGuarda:
                break

    temp_folder = [folder[i] for i in sicontiene]
    temp_filename = [filename[i] for i in sicontiene]
    return temp_folder, temp_filename

In [33]:


def responde(query):
    Menos10, top = seleccionaTop(query)
    cognitive, folder, filename = azure_cognitive_search(query, top) 

    if Menos10:
        try:
            _, respuesta = azure_openai(query, prompt_results=cognitive, isIntro=False)
            json_data = json.loads(respuesta)
            contenido = json_data["content"]
            folder, filename = selecciona_rutas(cognitive, contenido, folder, filename)
            return folder, filename, respuesta
        except Exception as e:
                error_message = "Lo sentimos, ha ocurrido un error en la ejecución.\nEstos son los resultados crudos de tu búsqueda"
                respuesta =json.dumps({"content": error_message})
                return folder, filename, respuesta
    else:
        error_message = "Por ahora puedo arrojar más de 10 resultados, intenta modificando tu solicitud.\nAquí tienes los archivos crudos de tu búsqueda"
        respuesta =json.dumps({"content": error_message})    
    
    return folder, filename, respuesta




# Tests

Integrate test

In [34]:
query = "Dame 4 perfiles con experiencia en DevOps y que sean de Global Hitss Cono Sur"
folder, filename, resp = responde(query)
json_data = json.loads(resp)

contenido = json_data["content"]
print(contenido)
separado = contenido.split("\n")    


['ALAN JAVIER CANELLAS', 'PUESTO ACTUAL:', 'EXPERIENCIA EN DEVOPS:', 'SOLIDO CONNECTING SOLUTIONS:', 'SOPORTE IT:', 'DIEGO JOSE GUILLEN BARRETO', 'PUESTO ACTUAL:', 'EXPERIENCIA EN DEVOPS:', 'GLOBAL HITSS CONO SUR:', 'LITEBOX ONLINE STRATEGIES:', 'FACUNDO VALENTIN CUPELLO', 'PUESTO ACTUAL:', 'EXPERIENCIA EN DEVOPS:', 'GLOBAL HITSS CONO SUR:', 'SCRUM MASTER:', 'MATIAS GERMAN PONCE', 'PUESTO ACTUAL:', 'EXPERIENCIA EN DEVOPS:', 'GLOBAL HITSS CONO SUR:', 'GLOBANT:']
Aquí tienes 4 perfiles de empleados de Global Hitss Cono Sur con experiencia en DevOps:

**Alan Javier Cañellas**
- **Puesto Actual:** Backend Developer
- **Experiencia en DevOps:**
  - **Solido Connecting Solutions:** Maquetación, estilizado y programación de aplicaciones web, trabajando en conjunto con el área de back-end. Mantenimiento y optimización de las mismas.
  - **Soporte IT:** Soporte técnico remoto y presencial de infraestructura de software para empresas varias, utilizando sistemas de gestión de tickets.

**Diego Jo

In [15]:
print(folder, filename)

['/Documentos compartidos/General/ARGENTINA', '/Documentos compartidos/General/ARGENTINA', '/Documentos compartidos/General/ARGENTINA', '/Documentos compartidos/General/ARGENTINA', '/Documentos compartidos/General/ARGENTINA', '/Documentos compartidos/General/ARGENTINA'] ['TOMAS MARTINHO.pdf', 'ALAN JAVIER CAÑELLAS.pdf', 'DIEGO SERGIO ANGEL.pdf', 'DIEGO JOSE GUILLEN BARRETO.pdf', 'FACUNDO VALENTIN CUPELLO.pdf', 'MATIAS GERMAN PONCE.pdf']


# tests by part

### Cognitive services document retrieval

In [18]:
query = "puedes darme perfiles con experiencia en SAP FI?"

In [19]:
Menos10, top = seleccionaTop(query)
print(Menos10, top)

True 5


In [20]:
cognitive, folder, filename = azure_cognitive_search(query, top) 

In [21]:
filename

['GUSTAVO ALBERTO DEYA.pdf',
 'DIEGO DE LA CRUZ ARROYO.pdf',
 'JULIAN MATIAS BERMOLEN.pdf',
 'DIEGO JOSE GUILLEN BARRETO.pdf',
 'DIEGO JOSE GUILLEN BARRETO.pdf']

In [41]:
parts = cognitive.split("****")
for part in parts:
    print(part)

Contenido del CV: AGUILAR PAZMIÑO DAVID ALEXIS Ing. Asdruval Conza Otuna 
095 879 0458
asdruval_94@hotmail.com
Ing. Ingrid Giraldo de Univ. Guayaquil 
0994885355
ingrid.giraldom@ug.edu.ec
Ing. Pazmiño Lucio Santiago de Claro
0996386783
spazmin0@claro.com.ecSoy una persona proactiva, puntual, responsable, con valores éticos y morales, lleno de energía y
vitalidad. Tengo experiencia en servicio al cliente, soporte y mantenimiento de dispositivos
electrónicos en general. Me adapto rápido a cualquier tipo de ambiente laboral y trato de acoplarme
fácilmente con el equipo de trabajo. Sé trabajar bajo presión, facilidad de palabra, rápido aprendizaje. 
COLEGIO FISCAL TECNICO PROVINCIA
DE BOLIVAR, GUAYAQUIL
BACHILLER CONTABLEEDUCACIÓN EXPERIENCIA LABORAL
Call center
Soporte técnico, recibir o realizar llamadas
para responder dudas y resolver cualquier
inconveniente.
Atencion de cliente servicio masivo.Telenews. Febrero 2021-Abril 2021
UNIVERSIDAD DE GUAYAQUIL 
INGENIERO EN NETWORKING Y
TELECOM

In [44]:
for i in range(len(folder)):
    print(folder[i], filename[i])

/Documentos compartidos/General/ECUADOR AGUILAR PAZMIÑO DAVID ALEXIS.pdf
/Documentos compartidos/General/ECUADOR PAZMIÑO SALAS JOSE ANTONIO.pdf
/Documentos compartidos/General/PERÚ DAVID STALING VALVERDE VALDIGLESIAS.pdf
/Documentos compartidos/General/ECUADOR SUAREZ AGUILAR CRISTOPHER DAVID.pdf
/Documentos compartidos/General/PruebaPDF DANIEL AGUILAR URIBE.pdf
/Documentos compartidos/General/PruebaPDF DAVID ULISSES ARENAS AGUILAR.pdf


### Open AI tests

In [45]:

_, respuesta = azure_openai(query, prompt_results=cognitive, isIntro=False)


json_data = json.loads(respuesta)
contenido = json_data["content"]

print(contenido)

El resumen del currículum de **David Aguilar Pazmiño** es el siguiente:

- Bachiller Contable del Colegio Fiscal Técnico Provincia de Bolívar, Guayaquil.
- Experiencia laboral en atención al cliente y soporte técnico en Telenews y Publinext S.A.
- Ingeniero en Networking y Telecomunicaciones de la Universidad de Guayaquil.
- Cursos y certificaciones en instalador de fibra óptica, fundamentos de Networking para redes IP, conectividad IP y protocolo de ruteo, técnico en seguridad informática, técnico en sistemas informáticos, entre otros.
- Habilidades en liderazgo en equipo, proactividad, resiliencia, paquete Microsoft Office, aplicaciones de seguridad, conocimiento en routers de marcas CISCO, TP-Link y HUAWEI.
- Nivel intermedio en idioma inglés.

Si necesitas más detalles sobre algún aspecto en particular, no dudes en preguntar.


In [46]:
folder, filename = selecciona_rutas(cognitive, contenido, folder, filename)
print(folder, filename)

['DAVID AGUILAR PAZMINO']
['/Documentos compartidos/General/ECUADOR', '/Documentos compartidos/General/ECUADOR'] ['AGUILAR PAZMIÑO DAVID ALEXIS.pdf', 'SUAREZ AGUILAR CRISTOPHER DAVID.pdf']
